In [13]:
# Creates new models into the DB without the need for an excel sheet
import sqlite3
import pandas as pd
import sys
sys.path.append('../')
from user_packages import hashing

conn = sqlite3.connect('full_metadata.db')
cur = conn.cursor()

In [14]:
# Set variables
# These are the things you want to create into the models hub

new_model_name = 'OH Physical Model' # A name for the model. This isn't 'magic', so you can call it what you want. Suggested: something relevant like 'x system physical model'.
new_model_concept_level = 'Physical' # One of:  Enterprise, Area, Conceptual, Logical, Physical
new_model_type = 'main' # One of: main, branch

In [18]:
# Collects a new KeyPhrase from the DB for the model
def GetNewKeyPhrase():
  sql_query = "SELECT NewKeyPhrase FROM NewKeyPhrase;";
  cur.execute(sql_query)
  row = cur.fetchone()
  NewKeyPhrase = row[0]
  #print(NewKeyPhrase)
  if NewKeyPhrase == '' : # if the view returns a blank row because it randomly selected a used ID
    NewKeyPhrase = GetNewKeyPhrase()
  sql_query = "INSERT INTO AppUsedId (AppUsedId) VALUES (?);";
  cur.execute(sql_query, [NewKeyPhrase])
  conn.commit()
  return NewKeyPhrase

In [20]:
data = [
  {
    'ModelKeyPhrase': GetNewKeyPhrase()
    , 'ModelName': new_model_name
    , 'ModelConceptLevel': new_model_concept_level
    , 'ModelType': new_model_type
  }
];
df = pd.DataFrame(data)

#df['ModelKeyPhrase'] = df.apply(lambda row: GetNewKeyPhrase() if pd.isnull(row['ModelKeyPhrase']) else row['ModelKeyPhrase'], axis=1)
#df.fillna('', inplace=True)

#df

OperationalError: no such table: NewKeyPhrase

In [4]:
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelHashKey'
  , columns = ['ModelKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = ['ModelName','ModelConceptLevel','ModelType']
)
df['RecordSource'] = df.apply(lambda row: 'Py.CreateModel', axis=1)

#df

In [5]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_CreateModel")
conn.commit()
df.to_sql('stg_Py_CreateModel', conn, if_exists='append', index=False)


3

In [6]:
# Write to the Hub
sql_query = """
INSERT INTO rv_h_Model
(
    ModelHashKey
  , LoadDate
  , RecordSource
  , ModelKeyPhrase
)
SELECT DISTINCT
    ModelHashKey
  , LoadDate
  , RecordSource
  , ModelKeyPhrase
FROM
  stg_Py_CreateModel
WHERE
  ModelHashKey NOT IN (SELECT ModelHashKey FROM rv_h_Model)
""";
conn.execute(sql_query)
conn.commit()

In [7]:
# Write to the Satellite
sql_query = """
INSERT INTO rv_s_Model
(
    ModelHashKey
  , LoadDate
  , RecordSource
  , HashDiff
  , ModelName
  , ModelConceptLevel
  , ModelType
)
SELECT DISTINCT
    stg.ModelHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  , stg.ModelName
  , stg.ModelConceptLevel
  , stg.ModelType
FROM
  stg_Py_CreateModel AS stg
  LEFT OUTER JOIN rv_s_Model AS sat ON (
    stg.ModelHashKey = sat.ModelHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_Model AS z
      WHERE z.ModelHashKey = sat.ModelHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()